# Footprint functionality
Here we show how to use the footprint functionality. It is not used directly in the matching, but can be applied on the recovery rates computation

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-random-data-and-add-to-catalog" data-toc-modified-id="Generate-random-data-and-add-to-catalog-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate random data and add to catalog</a></span></li><li><span><a href="#Add-an-external-footprint" data-toc-modified-id="Add-an-external-footprint-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add an external footprint</a></span><ul class="toc-item"><li><span><a href="#Plotting-the-footprint" data-toc-modified-id="Plotting-the-footprint-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Plotting the footprint</a></span></li></ul></li><li><span><a href="#Use-ClEvaR-functions-to-create-a-footprint" data-toc-modified-id="Use-ClEvaR-functions-to-create-a-footprint-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Use ClEvaR functions to create a footprint</a></span></li><li><span><a href="#Footprint-masks" data-toc-modified-id="Footprint-masks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Footprint masks</a></span></li><li><span><a href="#Saving-and-loading-footprint-quantities" data-toc-modified-id="Saving-and-loading-footprint-quantities-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saving and loading footprint quantities</a></span></li><li><span><a href="#Match-catalogs" data-toc-modified-id="Match-catalogs-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Match catalogs</a></span></li><li><span><a href="#Recovery-rate" data-toc-modified-id="Recovery-rate-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Recovery rate</a></span></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt
import healpy as hp

## Generate random data and add to catalog

In [ ]:
# For reproducibility
np.random.seed(1)

In [ ]:
from support import gen_cluster
input1, input2 = gen_cluster(ra_min=0, ra_max=30, dec_min=9, dec_max=30)

In [ ]:
from clevar import ClCatalog
c1 = ClCatalog('Cat1', ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'],
            mass_err=input1['MASS_ERR'], z_err=input1['Z_ERR'], radius=input1['RADIUS_ARCMIN'],
            radius_unit='arcmin')
c2 = ClCatalog('Cat2', ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'],
            mass_err=input2['MASS_ERR'], z_err=input2['Z_ERR'], radius=input2['RADIUS_ARCMIN'],
            radius_unit='arcmin')
# Format for nice display
for c in ('ra', 'dec', 'z', 'z_err', 'radius'):
    c1[c].info.format = '.2f'
    c2[c].info.format = '.2f'
for c in ('mass', 'mass_err'):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'

## Add an external footprint
Here we will get heapy pixels based on the positions of the clusters

In [ ]:
nside = 32
pixels1 = hp.ang2pix(nside, c1['ra'], c1['dec'], lonlat=True)
pixels2 = hp.ang2pix(nside, c2['ra'], c2['dec'], lonlat=True)

Check to see if selected pixels are correct

In [ ]:
from matplotlib import cm
import copy
cmap = copy.copy(cm.jet)
cmap.set_under('.1')
gcol = lambda cmap, level: '#{:02x}{:02x}{:02x}{:02x}'.format(*cmap(level,bytes=True))

# Map with pixels of each catalog
map_ = np.zeros(hp.nside2npix(nside))
map_[pixels1] += 1
map_[pixels2] += 2
map_[map_==0] = np.nan

f = plt.figure(figsize=(10, 10))
hp.cartview(map_, hold=True, latra=[5, 35], lonra=[-5, 40], cmap=cmap, cbar=False, flip='geo')
ax = f.axes[0]
ax.axis('on')
ax.scatter(c1['ra'], c1['dec'], s=5, label='Cat 1 clusters')
ax.scatter(c2['ra'], c2['dec'], s=5, label='Cat 2 clusters')

ax.plot(0, 0, zorder=0, color=gcol(cmap, 0.0), label='Footptint - Cat1 only')
ax.plot(0, 0, zorder=0, color=gcol(cmap, 0.5), label='Footptint - Cat2 only')
ax.plot(0, 0, zorder=0, color=gcol(cmap, 1.0), label='Footptint - BOTH')
ax.legend(loc=3)

Add them to the `Footprint` object. It also has an option of detection fraction and $z_{max}$ information. If not provided, a default value is set:

In [ ]:
from clevar.footprint import Footprint
# Random values for detfrac and zmax for ftpt1
detfrac_rand, z_rand = 0.9+.1*np.random.rand(len(set(pixels1))), 0.5+.5*np.random.rand(len(set(pixels1)))
ftpt1 = Footprint(nside=nside, pixels=list(set(pixels1)),
                  detfrac=detfrac_rand, zmax=z_rand)
ftpt2 = Footprint(nside=nside, pixels=list(set(pixels2)))

In [ ]:
f, axes = plt.subplots(1, 2)

bins = np.linspace(0.85, 1.05, 41)
axes[0].hist(ftpt1['detfrac'], bins=bins, label='ftpt1')
axes[0].hist(ftpt2['detfrac'], bins=bins, histtype='step', label='ftpt2')
axes[0].legend()
axes[0].set_xlabel('detfrac')
axes[0].set_yscale('log')

bins = np.append(np.linspace(0.5, 1.0, 11), [99, 100])
axes[1].hist(ftpt1['zmax'], bins=bins)
axes[1].hist(ftpt2['zmax'], bins=bins, histtype='step')
axes[1].set_xlabel('zmax')
axes[1].set_xscale('log')
axes[1].set_yscale('log')

### Plotting the footprint
The footprints have an inbuilt function to plot their values

In [ ]:
f = ftpt1.plot('detfrac', bad_val=np.nan, auto_lim=True)
f = ftpt1.plot('zmax', bad_val=np.nan, auto_lim=True)

Clusters can also be added to the plot with their actual angular size:

In [ ]:
f = ftpt1.plot('detfrac', bad_val=np.nan,
               ra_lim=[3, 8], dec_lim=[10, 15],
               cluster=c1)

## Use ClEvaR functions to create a footprint
Import `create_footprint` functions to create a footprint based on a cluster catalog.
It can create a footprint based on cluster positions with a given `NSIDE`, or compute the best `NSIDE` based on a cluster density per pixel. It also can fill holes in the footprint.

In [ ]:
from clevar.footprint import create_artificial_footprint

Fixed `NSIDE`:

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=64)

`NSIDE` from density:

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=None, min_density=4)

fill holes

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=64, neighbor_fill=5)

In [ ]:
f = ftpt1.plot('detfrac', np.nan,  latra=[5, 35], lonra=[-5, 40])

In [ ]:
ra, dec = hp.pix2ang(128, np.arange(hp.nside2npix(128)), lonlat=True)
ra.min(), ra.max(), dec.min(), dec.max()

## Footprint masks
Add masks to clusters regarding the footprint. The `ClCatalog` object has has 3 functions related to the footprint:
- `add_ftpt_masks`: info for cluster in footprint
- `add_ftpt_coverfrac`: computes cover fraction
- `add_ftpt_coverfrac_nfw2D`: computes cover fraction weighted by a project NFW profile

In [ ]:
%%time
c1.add_ftpt_masks(ftpt1, ftpt2)
c2.add_ftpt_masks(ftpt2, ftpt1)

In [ ]:
display(c1[:4])
display(c2[:4])

Add coverfraction values based on the footprint. It needs a cosmology object.

In [ ]:
from clevar.cosmology import AstroPyCosmology
cosmo = AstroPyCosmology()

In [ ]:
%%time
c1.add_ftpt_coverfrac(ftpt2, 1, 'mpc', cosmo=cosmo, window='flat')
c1.add_ftpt_coverfrac(ftpt2, 1, 'mpc', cosmo=cosmo, window='nfw2D')
c2.add_ftpt_coverfrac(ftpt1, 1, 'mpc', cosmo=cosmo, window='nfw2D')

In [ ]:
display(c1[:4])
display(c2[:4])

## Saving and loading footprint quantities
`ClEvaR` has internal functions to save and load these quantities into the catalog so you don't have to compute them again:

In [ ]:
c1.save_footprint_quantities('cat1_ft_quantities.fits', overwrite=True)
c1.load_footprint_quantities('cat1_ft_quantities.fits')

## Match catalogs

In [ ]:
from clevar.match import ProximityMatch

In [ ]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'},
    'catalog2': {'delta_z':.2,
                'match_radius': '10 arcsec'}
}

In [ ]:
%%time
mt = ProximityMatch()
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

## Recovery rate
Use pass the parameters `mask` (masks all clusters) or `mask_unmatched` (masks only unmatched clusters) to consider only specific clusters on the recovery rate.

In [ ]:
from clevar.match_metrics import recovery

In [ ]:
zbins = np.linspace(0, 2, 11)
mbins = np.logspace(13, 14, 5)

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 5))
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[0], add_legend=False)
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[1], add_legend=False,
              mask=c1.data['cf_nfw_1_mpc']<1)
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[2],
              mask_unmatched=c1.data['cf_nfw_1_mpc']<1)
for ax in axes:
    ax.set_ylim(-.01, 1.05)
axes[0].text(1, 1.1, 'no mask')
axes[1].text(1, 1.1, 'mask all')
axes[2].text(1, 1.1, 'mask unmatched')
plt.show()

You can check the exact numbers used on the 2D plots

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(20, 5))

recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[0],
                add_num=True, num_kwargs={'fontsize':12
                                         })
recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[1],
                add_num=True, num_kwargs={'fontsize':12},
               mask=c1.data['cf_nfw_1_mpc']<1)
recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[2],
                add_num=True, num_kwargs={'fontsize':12},
               mask_unmatched=c1.data['cf_nfw_1_mpc']<1)
axes[0].text(1, mbins[-1]*1.1,'no mask')
axes[1].text(1, mbins[-1]*1.1,'mask all')
axes[2].text(1, mbins[-1]*1.1,'mask unmatched')
    
plt.show()